In [1]:
import _init_paths
from datasets.factory import get_imdb
imdb = get_imdb('voc_2007_trainval')

In [20]:
a = imdb.roidb[0]

['gt_classes',
 'boxscores',
 'gt_vec',
 'flipped',
 'boxes',
 'seg_areas',
 'gt_overlaps']

In [21]:
import numpy as np
np.where(a['gt_classes']==0)[0]

array([   3,    4,    5, ..., 4000, 4001, 4002])

In [40]:
a.keys()

['gt_classes',
 'boxscores',
 'gt_vec',
 'flipped',
 'boxes',
 'seg_areas',
 'gt_overlaps']

In [33]:
np.where(a['gt_classes']!=0)[0]
a['gt_classes']
a['gt_overlaps'][8,8]
rois_blob = np.zeros((0, 5), dtype=np.float32)
print(rois_blob)

[]


In [44]:
len(imdb.classes)
imdb.classes
imdb.num_images
imdb.roidb[0]['gt_classes']
ss = imdb.gt_roidb()[2018]['gt_classes']
type(ss)

voc_2007_trainval gt roidb loaded from /home/yzn/CMU/18Spring/16824/hw2/hw2-release/code/data/cache/voc_2007_trainval_gt_roidb.pkl


numpy.ndarray

In [14]:
classes = imdb.classes
class_to_idx = {}
for item in classes:
    class_to_idx[item] = classes.index(item)
class_to_idx

{'aeroplane': 0,
 'bicycle': 1,
 'bird': 2,
 'boat': 3,
 'bottle': 4,
 'bus': 5,
 'car': 6,
 'cat': 7,
 'chair': 8,
 'cow': 9,
 'diningtable': 10,
 'dog': 11,
 'horse': 12,
 'motorbike': 13,
 'person': 14,
 'pottedplant': 15,
 'sheep': 16,
 'sofa': 17,
 'train': 18,
 'tvmonitor': 19}

In [20]:
# len(imdb.image_index)
img_name = imdb.image_path_at(2018)
img_name

'/home/yzn/CMU/18Spring/16824/hw2/hw2-release/code/data/VOCdevkit2007/VOC2007/JPEGImages/003998.jpg'

In [16]:
import cv2
import numpy as np
import visdom
def vis_detections(im, class_name, dets, thresh=0.8):
    """Visual debugging of detections."""
    for i in range(np.minimum(10, dets.shape[0])):
        bbox = tuple(int(np.round(x)) for x in dets[i, :4])
        score = dets[i, -1]
        if score > thresh:
            cv2.rectangle(im, bbox[0:2], bbox[2:4], (0, 204, 0), 2)
            cv2.putText(im, '%s: %.3f' % (class_name, score), (bbox[0], bbox[1] + 15), cv2.FONT_HERSHEY_PLAIN,
                        1.0, (0, 0, 255), thickness=1)
    return im


In [73]:
gt_box = imdb.gt_roidb()

voc_2007_trainval gt roidb loaded from /home/yzn/CMU/18Spring/16824/hw2/hw2-release/code/data/cache/voc_2007_trainval_gt_roidb.pkl


In [88]:
print(gt_box[2]['gt_classes'])
from matplotlib.pyplot import imshow
from PIL import Image
%matplotlib inline
with open(imdb.image_path_at(2), 'rb') as f:
    img = Image.open(f)
    img.convert('RGB')
    img.show()

[13 15 15 15]


In [18]:
gt_box

{'boxes': array([[ 11,  59, 499, 306]], dtype=uint16),
 'flipped': False,
 'gt_classes': array([7], dtype=int32),
 'gt_overlaps': <1x21 sparse matrix of type '<type 'numpy.float32'>'
 	with 1 stored elements in Compressed Sparse Row format>,
 'seg_areas': array([ 121272.], dtype=float32)}

In [8]:
img = cv2.imread(img_name)
res = vis_detections(img, 'car', gt_box['boxes'])


In [9]:
res.shape

(375, 500, 3)

In [10]:
# cv2.imshow('car',res)
# cv2.waitKey()

In [14]:
res.transpose(2,1,0).shape

(3, 500, 375)

In [13]:
res.shape

(375, 500, 3)

In [15]:
ss_box = imdb.selective_search_roidb()[2018]
ss_box['boxscores'][-10:]

voc_2007_trainval ss roidb loaded from /home/yzn/CMU/18Spring/16824/hw2/hw2-release/code/data/cache/voc_2007_trainval_selective_search_roidb.pkl


In [17]:
ss_res = vis_detections(img, 'stuff', ss_box['boxes'][-10:])
cv2.imshow('car',ss_res)
cv2.waitKey()


KeyboardInterrupt: 

In [45]:
import torch.utils.data as data
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
model_urls = {
        'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}

from PIL import Image
import os
import os.path
import numpy as np

In [46]:
model_dict = model_zoo.load_url(model_urls['alexnet'])

In [48]:
model_dict.keys()

[u'features.0.weight',
 u'features.0.bias',
 u'features.3.weight',
 u'features.3.bias',
 u'features.6.weight',
 u'features.6.bias',
 u'features.8.weight',
 u'features.8.bias',
 u'features.10.weight',
 u'features.10.bias',
 u'classifier.1.weight',
 u'classifier.1.bias',
 u'classifier.4.weight',
 u'classifier.4.bias',
 u'classifier.6.weight',
 u'classifier.6.bias']

In [51]:
d = [x for x in model_dict.keys() if 'features' in x]

In [53]:
features_dict = {k: v for k, v in model_dict.items() if 'features' in k}

In [55]:
features_dict.keys()

[u'features.10.bias',
 u'features.10.weight',
 u'features.6.bias',
 u'features.6.weight',
 u'features.8.weight',
 u'features.3.bias',
 u'features.8.bias',
 u'features.0.weight',
 u'features.0.bias',
 u'features.3.weight']

In [56]:
import torch.utils.data as data
import torch.nn as nn
import torch.utils.model_zoo as model_zoo
import torch.nn.functional as F

In [67]:
class LocalizerAlexNet(nn.Module):
    def __init__(self, num_classes=20):
        super(LocalizerAlexNet, self).__init__()
        #TODO: Define model
        self.features = nn.Sequential(
                            nn.Conv2d(3, 64, 11, stride=4, padding=2),
                            nn.ReLU(),
                            nn.MaxPool2d(3, stride=2),
                            nn.Conv2d(64, 192, 5, stride=1, padding=2),
                            nn.ReLU(),
                            nn.MaxPool2d(3, stride=2),
                            nn.Conv2d(192, 384, 3, stride=1, padding=1),
                            nn.ReLU(),
                            nn.Conv2d(384, 256, 3, stride=1, padding=1),
                            nn.ReLU(),
                            nn.Conv2d(256, 256, 3, stride=1, padding=1),
                            nn.ReLU()
            )

        self.classifier = nn.Sequential(
                        nn.Conv2d(256, 256, 3, stride=1),
                        nn.ReLU(),
                        nn.Conv2d(256, 256, 1, stride=1),
                        nn.ReLU(),
                        nn.Conv2d(256, num_classes, 1, stride=1)
            )


#         self.conv_1 = nn.Conv2d(3, 64, 11, stride=4, padding=2)
#         self.conv_2 = nn.Conv2d(64, 192, 5, stride=1, padding=2)
#         self.conv_3 = nn.Conv2d(192, 384, 3, stride=1, padding=1)
#         self.conv_4 = nn.Conv2d(384, 256, 3, stride=1, padding=1)
#         self.conv_5 = nn.Conv2d(256, 256, 3, stride=1, padding=1)

#         self.conv6 = nn.Conv2d(256, 256, 3, stride=1)
#         self.conv7 = nn.Conv2d(256, 256, 1, stride=1)
#         self.conv8 = nn.Conv2d(256, num_classes, 1, stride=1)

    def forward(self, x):
        #TODO: Define forward pass
        features = self.features(x)
        return self.classifier(features)

#         x = F.max_pool2d(F.relu(self.conv1(x)), 3, stride=2)
#         x = F.max_pool2d(F.relu(self.conv2(x)), 3, stride=2)
#         x = F.relu(self.conv3(x))
#         x = F.relu(self.conv4(x))
#         x = F.relu(self.conv5(x))
#         x = F.relu(self.conv6(x))
#         x = F.relu(self.conv7(x))
#         return self.conv8(x)

In [6]:
import argparse
import os
import shutil
import time
import sys
sys.path.insert(0,'/home/spurushw/reps/hw-wsddn-sol/faster_rcnn')
import sklearn
import sklearn.metrics

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import _init_paths
from datasets.factory import get_imdb
from custom import *
from logger import *
trainval_imdb = get_imdb('voc_2007_trainval')

In [8]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
train_dataset = IMDBDataset(
        trainval_imdb,
        transforms.Compose([
            transforms.Resize((512,512)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ]))

voc_2007_trainval gt roidb loaded from /home/yzn/CMU/18Spring/16824/hw2/hw2-release/code/data/cache/voc_2007_trainval_gt_roidb.pkl


In [10]:
train_sampler = None
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, shuffle=(train_sampler is None),
    num_workers=4, pin_memory=True, sampler=train_sampler)


In [14]:
train_loader.sampler

In [16]:
print("ouer", 123)

('ouer', 123)
